## Ceate Dataset

In [1]:
import torch
from torch.utils.data import Dataset

import numpy as np
import csv


class StockDataset(Dataset):
    ''' Dataset for loading and preprocessing the dataset '''
    def __init__(self,
                 path,
                 mode='train'
              ):
        self.mode = mode

        # Read data into numpy arrays
        with open(path, 'r') as fp:
          data = list(csv.reader(fp))
          data = np.array(data[1:])[:, 4:].astype(float)
        
        target = data[:, -1]
        data = data[:, list(range(22))]

        if mode == 'test':
            # Testing data
            self.data = torch.FloatTensor(data)
            self.target = torch.LongTensor(target)
        else:
            # Training data (train/dev sets)
        
            # Splitting training data into train & dev sets
            gap = int(len(data) * 0.8)
            if mode == 'train':
                
                self.data = torch.FloatTensor(data[:gap])
                self.target = torch.LongTensor(target[:gap])
            elif mode == 'dev':
                            
            # Convert data into PyTorch tensors
                self.data = torch.FloatTensor(data[gap:])
                self.target = torch.LongTensor(target[gap:])

        # Normalize features
        self.data = \
            (self.data - self.data.mean(dim=0, keepdim=True)) \
            / self.data.std(dim=0, keepdim=True)

        self.dim = self.data.shape[1]

        # print(f'Finished reading the {mode} set of Dataset ({len(self.data)} samples found, each dim = {self.dim})')

    def __getitem__(self, index):

        return self.data[index], self.target[index]

    def __len__(self):

        return len(self.data)


## Create Model

In [2]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.layer1 = nn.Linear(22, 512)
        self.layer1_bn=nn.BatchNorm1d(512)

        self.layer2 = nn.Linear(512, 256)
        self.layer2_bn=nn.BatchNorm1d(256)

        self.layer3 = nn.Linear(256, 128)
        self.layer3_bn=nn.BatchNorm1d(128)

        self.layer4 = nn.Linear(128, 64)
        self.layer4_bn=nn.BatchNorm1d(64)

        self.layer5 = nn.Linear(64, 32)
        self.layer5_bn=nn.BatchNorm1d(32)

        # self.drop = nn.Dropout(0.5)

        self.out = nn.Linear(32, 3) 

        # self.act_fn = nn.Sigmoid()
        self.act_fn = nn.ReLU()
        

    def forward(self, x):
        # x = self.drop(x)

        x = self.layer1(x)
        x = self.layer1_bn(x)
        x = self.act_fn(x)
        # x = self.drop(x)

        x = self.layer2(x)
        x = self.layer2_bn(x)
        x = self.act_fn(x)
        # x = self.drop(x)

        x = self.layer3(x)
        x = self.layer3_bn(x)
        x = self.act_fn(x)
        # x = self.drop(x)
 
        x = self.layer4(x)
        x = self.layer4_bn(x)
        x = self.act_fn(x)
        # x = self.drop(x)

        x = self.layer5(x)
        x = self.layer5_bn(x)
        x = self.act_fn(x)
        # x = self.drop(x)

        x = self.out(x)

        
        return x

In [3]:

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.ioff()
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# fix random seed for reproducibility
same_seeds(0)

# get device 
device = 'cpu'
print(f'DEVICE: {device}')

# training parameters
num_epoch = 400        # number of training epoch

learning_rate = 0.001       # learning rate


def macro_precision(y_true, y_predict):
    hit_count = {0: 0, 1: 0, 2: 0}

    for i, prediction in enumerate(y_predict):
        if y_true[i] == prediction:
            hit_count[prediction] += 1
    
    # r1 = hit_count[0]/y_true.count(0)
    # r2 = hit_count[1]/y_true.count(1)
    # r3 = hit_count[2]/y_true.count(2)

    p1 = hit_count[0]/y_predict.count(0)
    p2 = hit_count[1]/y_predict.count(1)
    p3 = hit_count[2]/y_predict.count(2)

    # f1 = 2 * p1 * r1 / (p1 + r1)
    # f2 = 2 * p2 * r2 / (p2 + r2)
    # f3 = 2 * p3 * r3 / (p3 + r3)
    
    return (p1 + p2 + p3) / 3

def plot_learning_curve(acc_record, title, y_label, i , d):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_epoch = len(acc_record['train'])
    x = range(total_epoch)
    # x_2 = x_1[::len(acc_record['train']) // len(acc_record['dev'])]
    figure(figsize=(6, 4))
    plt.plot(x, acc_record['train'], c='tab:red', label='train')
    plt.plot(x, acc_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.0, 1.0)
    plt.xlabel('Training Epoch')
    plt.ylabel(y_label)
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.savefig(f'./models/fig/{y_label}_{i}_{d}d.png')
    plt.close()

DEVICE: cpu


## Training / Testing

In [4]:
BATCH_SIZE = 32

from torch.utils.data import DataLoader
from sklearn.metrics import f1_score  

error_list = []
for i in range(1, 201):
    print(f'Model {i}')
    for d in [15, 30, 90, 180]:
        print(f'{d}d')
        try:
            train_set = StockDataset(f'./data/training_data/training_data_{i}_{d}d.csv', mode = 'test')
            val_set = StockDataset(f'./data/training_data/testing_data_{i}_{d}d.csv', mode = 'test')
            train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
            val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

            # the path where checkpoint saved
            model_path = f'./models/models/model_{i}_{d}d.ckpt'

            # create model, define a loss function, and optimizer
            model = Classifier().to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            # Training

            best_acc = 0.0
            best_macro_precision = 0.0

            acc_record = {'train': [], 'dev': []}
            macro_f1_record = {'train': [], 'dev': []}
            macro_precision_score_record = {'train': [], 'dev': []}

            # init accuracy
            train_acc = 0.0
            val_acc = 0.0

            train_labels = []
            train_predictions = []
            val_labels = []
            val_predictions = []
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, val_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item() 
                    for y in val_pred.cpu().numpy():
                        val_predictions.append(y)

                    for y in labels.cpu().numpy():
                        val_labels.append(y)
                
                for data in train_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs) 
                    _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    train_acc += (train_pred.cpu() == labels.cpu()).sum().item() 
                    for y in train_pred.cpu().numpy():
                        train_predictions.append(y)

                    for y in labels.cpu().numpy():
                        train_labels.append(y)

                acc_record['dev'].append(val_acc/len(val_set))
                acc_record['train'].append(train_acc/len(train_set))
                macro_f1_record['dev'].append(f1_score(val_labels, val_predictions, average='macro'))
                macro_f1_record['train'].append(f1_score(train_labels, train_predictions, average='macro'))
                macro_precision_score_record['dev'].append(macro_precision(val_labels, val_predictions))
                macro_precision_score_record['train'].append(macro_precision(train_labels, train_predictions))

            # start training
            for epoch in range(num_epoch):
            
                train_acc = 0.0
                train_loss = 0.0
                val_acc = 0.0
                val_loss = 0.0

                train_labels = []
                train_predictions = []
                val_labels = []
                val_predictions = []

                # training
                model.train() # set the model to training mode
                for data in train_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad() 
                    outputs = model(inputs) 
                    batch_loss = criterion(outputs, labels)
                    _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    batch_loss.backward() # compute gradient
                    optimizer.step() # update model with optimizer

                    train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
                    train_loss += batch_loss.item()
                    for y in train_pred.cpu().numpy():
                        train_predictions.append(y)

                    for y in labels.cpu().numpy():
                        train_labels.append(y)

                
                acc_record['train'].append(train_acc/len(train_set))

                train_f1 = f1_score(train_labels, train_predictions, average='macro')
                macro_f1_record['train'].append(train_f1)

                train_weighted_precision = macro_precision(train_labels, train_predictions)
                macro_precision_score_record['train'].append(train_weighted_precision)

                # validation

                model.eval() # set the model to evaluation mode
                with torch.no_grad():
                    for data in val_loader:
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        batch_loss = criterion(outputs, labels) 
                        _, val_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    
                        val_acc += (val_pred.cpu() == labels.cpu()).sum().item() 
                        val_loss += batch_loss.item()
                        for y in val_pred.cpu().numpy():
                            val_predictions.append(y)

                        for y in labels.cpu().numpy():
                            val_labels.append(y)


                    acc_record['dev'].append(val_acc/len(val_set))

                    val_f1 = f1_score(val_labels, val_predictions, average='macro')
                    macro_f1_record['dev'].append(val_f1)

                    val_weighted_precision = macro_precision(val_labels, val_predictions)
                    macro_precision_score_record['dev'].append(val_weighted_precision)


                    # if the model improves, save a checkpoint at this epoch
                    if val_weighted_precision > best_macro_precision:
                        best_macro_precision = val_weighted_precision
                        torch.save(model.state_dict(), model_path)
                        # print('[{:03d}/{:03d}] Train Acc: {:3.6f} F1: {:.3f} wP: {:.3f} Loss: {:3.6f} | \
                        # Val Acc: {:3.6f} F1: {:.3f} wP: {:.3f} loss: {:3.6f}'
                            # .format(epoch + 1, num_epoch, train_acc/len(train_set), train_f1, train_weighted_precision, train_loss/len(train_loader), 
                            #                             val_acc/len(val_set), val_f1, val_weighted_precision, val_loss/len(val_loader)))
                        # print('saving model with Val Acc {:.3f}'.format(best_acc/len(val_set)))
                        # print('saving model with Val Precision {:.3f}'.format(best_macro_precision))
                        early_stop_cnt = 0
                    else:
                        early_stop_cnt += 1

            plot_learning_curve(acc_record, 'deep model', 'Accuracy', i, d)
            plot_learning_curve(macro_precision_score_record, 'deep model', 'Macro Precision', i, d)
            plot_learning_curve(macro_f1_record, 'deep model', 'Macro F1 Score', i, d)
        except Exception as e:
            error_list.append([i, d, str(e)])
            


Model 1
15d
30d
90d
180d
Model 2
15d
30d
90d
180d
Model 3
15d
30d
90d
180d
Model 4
15d
30d
90d
180d
Model 5
15d
30d
90d
180d
Model 6
15d
30d
90d
180d
Model 7
15d
30d
90d
180d
Model 8
15d
30d
90d
180d
Model 9
15d
30d
90d
180d
Model 10
15d
30d
90d
180d
Model 11
15d
30d
90d
180d
Model 12
15d
30d
90d
180d
Model 13
15d
30d
90d
180d
Model 14
15d
30d
90d
180d
Model 15
15d
30d
90d
180d
Model 16
15d
30d
90d
180d
Model 17
15d
30d
90d
180d
Model 18
15d
30d
90d
180d
Model 19
15d
30d
90d
180d
Model 20
15d
30d
90d
180d
Model 21
15d
30d
90d
180d
Model 22
15d
30d
90d
180d
Model 23
15d
30d
90d
180d
Model 24
15d
30d
90d
180d
Model 25
15d
30d
90d
180d
Model 26
15d
30d
90d
180d
Model 27
15d
30d
90d
180d
Model 28
15d
30d
90d
180d
Model 29
15d
30d
90d
180d
Model 30
15d
30d
90d
180d
Model 31
15d
30d
90d
180d
Model 32
15d
30d
90d
180d
Model 33
15d
30d
90d
180d
Model 34
15d
30d
90d
180d
Model 35
15d
30d
90d
180d
Model 36
15d
30d
90d
180d
Model 37
15d
30d
90d
180d
Model 38
15d
30d
90d
180d
Model 39
15d
30d
90d


In [6]:
len(error_list)

36

In [5]:
error_list

[[3,
  30,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [4,
  180,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [8,
  90,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [11,
  15,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [13,
  180,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [16, 180, 'division by zero'],
 [22,
  180,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [27,
  90,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [31,
  90,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512])'],
 [46,
  180,
  'Expected more than 1 value per channel when training, got input size torch.Size([1, 512]